In [4]:
import math
import time
import os
import cv2 
import keyboard
import pickle
import numpy as np
from PIL import Image, ImageOps, ImageGrab
from matplotlib import pyplot as plt
from skimage import io, color, measure, morphology
from scipy import ndimage
from scipy.ndimage import rotate
import copy
%matplotlib inline

In [5]:
# Load the player_circle_center from the file.
with open('player_circle_center_1.pkl', 'rb') as file:
    player_circle_center = pickle.load(file)
with open('sscoords.pkl', 'rb') as file:
    sscoords = pickle.load(file)
with open('chatcoords.pkl', 'rb') as file:
    chatcoords = pickle.load(file)

furthest_point_distance = np.sqrt((player_circle_center[0])**2 + (player_circle_center[1])**2) # Distance from player to corner of screen. This will be used to measure how far away enemies and items are.

In [3]:
# Returns the grid of the inserted pixel.
def GetPosition(original_shape, resized_shape, pixel_pos):
    x = 4
    y = 4

    shape_diff_x = resized_shape[0] - original_shape[0]
    shape_diff_y = resized_shape[1] - original_shape[1]
    
    fifth_x = original_shape[0]/5
    fifth_y = original_shape[1]/5
    
    for i in range(1, 5):
        if pixel_pos[0] < (shape_diff_x/2) + (fifth_x*i):
            x = i-1
            break
        
    for i in range(1, 5):
        if pixel_pos[1] < (shape_diff_y/2) + (fifth_y*i):
            y = i-1
            break
    return x, y

In [17]:
def GetEnemiesAndItems(original_img):
    image_np = np.array(original_img)
    
    # Sizing Img Down
    scale_fraction = 0.4
    width = int(image_np.shape[1] * scale_fraction)
    height = int(image_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_np = cv2.resize(image_np, dim)

    image_player_np = copy.deepcopy(image_np)

    # Assuming image_player_np is your image numpy array and player_circle_center is a tuple (x, y)
    height, width = image_player_np.shape[:2]
    # Create a grid of indices
    y_indices, x_indices = np.indices((height, width))
    # Calculate squared distances from the player center
    distances_squared = (x_indices - player_circle_center[1])**2 + (y_indices - player_circle_center[0])**2
    # Create a mask for pixels within 10 pixels of the player center
    near_player_mask = distances_squared <= 10**2
    # Apply the mask to set near player pixels to black
    image_player_np[near_player_mask] = 0
    # Mask for the color condition
    color_condition = (image_player_np[:, :, 2] > 100) & (image_player_np[:, :, 2] > image_player_np[:, :, 1] * 1.4) & (image_player_np[:, :, 0] < 20)
    # Combine the color condition with the distance condition
    final_mask = ~near_player_mask & color_condition
    # Apply the mask to set the specified pixels to white, and others to black
    image_player_np[final_mask] = [255, 255, 255]
    image_player_np[~final_mask] = [0, 0, 0]
    
    player_image = cv2.cvtColor(image_player_np, cv2.COLOR_BGR2GRAY)

    _, player_binary_image = cv2.threshold(player_image, 100, 255, cv2.THRESH_BINARY)

    player_labeled_mask, _ = ndimage.label(player_binary_image)

    player_regions = measure.regionprops(player_labeled_mask)
    
    min_area = 200
    
    closest_region_dist = np.inf
    closest_region_center = (0, 0)
    
    # Loop through region properties and filter out non-circular regions
    for region in player_regions:
        if region.area > min_area and region.perimeter > 0:
            centroid = region.centroid
            region_dist = np.sqrt((centroid[0] - player_circle_center[0])**2 + (centroid[1] - player_circle_center[1])**2)
            if region_dist < closest_region_dist:
                closest_region_dist = region_dist
                closest_region_center = centroid

    # Getting the view area center relative to the player.
    relative_center = (player_circle_center[0] - closest_region_center[0], closest_region_center[1] - player_circle_center[1])

    # Measuring the angle the player is looking
    angle_radians = math.atan2(relative_center[0], relative_center[1])
    player_direction_angle = math.degrees(angle_radians)
    if player_direction_angle < 0:
        player_direction_angle += 360

    # ################## ENEMIES SECTION ##################
    rotated_img = rotate(image_np, angle=-(player_direction_angle-90), reshape=True, mode='constant', cval=0)
    image_enemy_np = copy.deepcopy(rotated_img)

    # Highlighting the red pixels.
    red_condition = (image_enemy_np[:, :, 0] > 100) & (image_enemy_np[:, :, 0] > image_enemy_np[:, :, 1]*2) & (image_enemy_np[:, :, 0] > image_enemy_np[:, :, 2]*2) & (image_enemy_np[:, :, 1] < 80)
    image_enemy_np[red_condition] = [255, 255, 255]
    image_enemy_np[~red_condition] = [0, 0, 0]
    
    image = cv2.cvtColor(image_enemy_np, cv2.COLOR_BGR2GRAY)
    
    # Check if the image is already in binary form, if not, apply a threshold
    # Assuming white pixel values are close to 255
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)
    
    # Label the objects in the binary image
    labeled_mask, _ = ndimage.label(binary_image)
    
    # Find the properties of the labeled regions
    regions = measure.regionprops(labeled_mask)
    
    # Define a range for the circularity metric (0 for a line, 1 for a perfect circle)
    circularity_threshold = 0.3
    
    min_radius = 3.5
    
    regions_centroids = []
    
    # Loop through region properties and filter out non-circular regions
    for region in regions:
        if region.area > 0 and region.perimeter > 0:
            # To measure the circularity of the region, we'll do the following:
            # Step 1, get all the points in the region.
            centroid = region.centroid
            coords = np.array(region.coords)
    
            # Step 2: Get the furthest point from the center.
            biggest_r = 0
            furthest_point = ()
            for coord in coords:
                distance = abs(centroid[0] - coord[0]) + abs(centroid[1] - coord[1])
                if distance > biggest_r:
                    biggest_r = distance
                    furthest_point = coord
    
            # Step 3: Compare the region area with a perfect circle area.
            circle_area = np.pi * (biggest_r ** 2)
    
            # Step 4: Calculate circularity
            circularity = region.area/circle_area
            
            # If the region is within the circularity thresholds and has at least the minimum diameter, keep it
            if circularity_threshold <= circularity and biggest_r >= min_radius:
                regions_centroids.append(region.centroid)

    enemies = []
    
    for centroid in regions_centroids:
        enemies.append(GetPosition(image_np.shape, image_enemy_np.shape, centroid))

    # ################## ITEMS SECTION ##################
    pil_rotated_img = Image.fromarray(rotated_img)
    
    posterized_image = ImageOps.posterize(pil_rotated_img, 2)
    
    image_item_np = np.array(posterized_image)

    # Sizing Img Down
    scale_fraction = 0.5
    width = int(image_item_np.shape[1] * scale_fraction)
    height = int(image_item_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_item_np = cv2.resize(image_item_np, dim)

    # Highlighting the yellow pixels.
    yellow_condition = (image_item_np[:, :, 0] > 150) & (image_item_np[:, :, 1] > 150) & (image_item_np[:, :, 2] < 15) & (abs(image_item_np[:, :, 0] - image_item_np[:, :, 1]) < 15)
    image_item_np[yellow_condition] = [255, 255, 255]
    image_item_np[~yellow_condition] = [0, 0, 0]
    
    # Removing small bits.
    deletion_range = (7, 7)
    size_threshold = 7 # The number of active pixels that have to be within the range around an active pixel.
    
    # Get the coordinates of the white pixels
    white_pixels_mask = (image_item_np[:, :, 0] == 255) & (image_item_np[:, :, 1] == 255) & (image_item_np[:, :, 2] == 255)
    white_pixel_coords = np.argwhere(white_pixels_mask)

    # Iterate over each white pixel's coordinates and removing lone pixels.
    for x, y in white_pixel_coords:
        # Count the number of white pixels in the surrounding area
        active_neighbors = np.sum((abs(white_pixel_coords[:, 0] - x) < deletion_range[0]) & (abs(white_pixel_coords[:, 1] - y) < deletion_range[1]))
        # If the number of white neighbors is less than the minimum required, turn the pixel black
        if active_neighbors < size_threshold:
            image_item_np[x, y] = (0, 0, 0)  # Turn the pixel black
    
    image_item = cv2.cvtColor(image_item_np, cv2.COLOR_BGR2GRAY)

    # Check if the image is already in binary form, if not, apply a threshold
    _, binary_image = cv2.threshold(image_item, 100, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    binary_image = cv2.dilate(binary_image, kernel, iterations=1)

    # Label the objects in the binary image
    labeled_mask, _ = ndimage.label(binary_image)
    
    # Find the properties of the labeled regions
    items_regions = measure.regionprops(labeled_mask)
    
    items = []
    half_shape = (image_np.shape[0] * scale_fraction, image_np.shape[1] * scale_fraction, 3)
    
    for region in items_regions:
        centroid = region.centroid
        items.append(GetPosition(half_shape, image_item_np.shape, centroid))
    
    return enemies, items

In [6]:
standards = []
standards.append(np.load("first.npy"))
standards.append(np.load("second.npy"))
standards.append(np.load("third.npy"))

def GetChatPromptCode(original_img):
    image_np = np.array(original_img)
    sections = []

    sections.append(image_np[:, :16, :])
    sections.append(image_np[:, 16:32, :])
    sections.append(image_np[:, 32:, :])

    command = ""

    for section, standard in zip(sections, standards):
        similar_pixels = 0
        for i in range(section.shape[0]):  # Iterate over rows (height)
            for j in range(section.shape[1]):  # Iterate over columns (width)
                r, g, b = section[i, j]
                r = int(r)
                g = int(g)
                b = int(b)
    
                if abs(r - g) < 20 and abs(b - r) > 40 and r > 70 and b < 100:
                    if standard[i,j] == 1:
                        similar_pixels += 1
                    else:
                        similar_pixels -= 1
                else:
                    if standard[i,j] == 1:
                        similar_pixels -= 1
        similarity = max(0, similar_pixels/np.sum(standard == 1))
        if similarity > 0.7:
            command += "Z"
        else:
            command += " "
                    
    return command

In [7]:
from gtts import gTTS
import uuid
from playsound import playsound
import pyautogui

def text_to_speech(text):
    # Create a gTTS object
    tts = gTTS(text=text, lang='en')
    filename = "output_{}.mp3".format(str(uuid.uuid1()))

    # Save the speech to an MP3 file
    tts.save(filename)

    pyautogui.mouseDown()
    # Play the audio file (this will play through the default audio device)
    playsound(filename)
    pyautogui.mouseUp()
    
    # Clean up
    os.remove(filename)

In [8]:
def waitFor(desired_duration):
    start_time = time.perf_counter()
    while True:
        current_time = time.perf_counter()
        if current_time - start_time >= desired_duration:
            break

In [9]:
import pyperclip
def switchTo(uname):
    comnd = "switch {}".format(uname)
    pyperclip.copy(comnd)
    pyautogui.hotkey('ctrl', 'v')
    waitFor(0.1)
    pyautogui.press('enter')

def flashBooster(booster_name):
    comnd = "flash {}".format(booster_name)
    pyperclip.copy(comnd)
    pyautogui.hotkey('ctrl', 'v')
    waitFor(0.1)
    pyautogui.press('enter')
    waitFor(0.1)
    pyautogui.press('d')
    waitFor(0.1)
    pyautogui.press('enter')
    waitFor(0.1)

def getDropShip():
    comnd = "walkie"
    pyperclip.copy(comnd)
    pyautogui.hotkey('ctrl', 'v')
    waitFor(0.1)
    pyautogui.press('enter')
    waitFor(0.1)
    pyautogui.press('c')
    waitFor(0.1)
    pyautogui.press('enter')
    waitFor(0.1)

def pdi_press(key):
    pydirectinput.keyDown(key)
    waitFor(0.1)  # Hold the key for a short duration
    pydirectinput.keyUp(key)

def sendMessage(msgs):
    for msg in msgs:
        if msg == "":
            continue
            
        pdi_press('/')
        
        pyperclip.copy(msg)
        pyautogui.hotkey('ctrl', 'v')
        waitFor(0.1)

        pdi_press('enter')
        waitFor(0.1)

    pdi_press('e')
    waitFor(1)

    switchTo("SyntaxeAI")
    waitFor(0.5)

def walkBack():
    pydirectinput.keyDown('s')
    waitFor(0.2)  # Hold the key for a short duration
    pydirectinput.keyUp('s')

In [10]:
import pydirectinput
import vgamepad as vg

# Optional: Fail-safes to prevent your script from running amok
pydirectinput.FAILSAFE = True

In [11]:
def fix_joystick():
    # Create a virtual gamepad (Xbox 360 type by default)
    gamepad = vg.VX360Gamepad()
    waitFor(4)
    
    # Move right joystick to the right (X-axis maximum, Y-axis neutral)
    gamepad.right_joystick(x_value=10000, y_value=0)
    gamepad.update()

    # Wait for a bit
    waitFor(2)

    # Reset joystick to neutral position
    gamepad.right_joystick(x_value=0, y_value=0)
    gamepad.update()
    waitFor(0.1)

    gamepad.right_joystick(x_value=-10000, y_value=0)
    gamepad.update()

    # Wait for a bit
    waitFor(2)

    # Reset joystick to neutral position
    gamepad.right_joystick(x_value=0, y_value=0)
    gamepad.update()
    waitFor(0.1)

    # Close the gamepad
    gamepad.reset()
    gamepad.update()

    waitFor(0.1)
    pdi_press('esc')

In [12]:
def charge_walkie():
    # Create a virtual gamepad (Xbox 360 type by default)
    gamepad = vg.VX360Gamepad()
    waitFor(4)
    # Move right joystick to the right (X-axis maximum, Y-axis neutral)
    gamepad.right_joystick(x_value=10000, y_value=0)
    gamepad.update()

    # Wait for a bit
    waitFor(0.8)

    # Reset joystick to neutral position
    gamepad.right_joystick(x_value=0, y_value=0)
    gamepad.update()
    waitFor(0.1)

    pdi_press('e')
    waitFor(4)

    gamepad.right_joystick(x_value=-10000, y_value=0)
    gamepad.update()

    # Wait for a bit
    waitFor(1.1)

    # Reset joystick to neutral position
    gamepad.right_joystick(x_value=0, y_value=0)
    gamepad.update()
    waitFor(0.1)

    # Close the gamepad
    gamepad.reset()
    gamepad.update()


In [13]:
def GetStrings(enemies, items):
    count_str = "E: {} . I: {}".format(len(enemies), len(items))
    map_str1 = count_str
    map_str2 = ""
    enemies_arr = np.zeros((5, 5))
    for e in enemies:
        enemies_arr[e[0]][e[1]] = 1
    
    items_arr = np.zeros((5, 5))
    for i in items:
        items_arr[i[0]][i[1]] = 1
    
    for i in range(5):
        row = "\n"
        for j in range(5):
            if enemies_arr[i][j] == 1 and items_arr[i][j] == 1:
                row += "Z"
            elif enemies_arr[i][j] == 1:
                row += "X"
            elif items_arr[i][j] == 1:
                row += "I"
            else:
                row += "O"
        if i < 2:
            map_str1 += row
        else:
            map_str2 += row
        
    return [map_str1, map_str2]

### Instructions:
    1- In game, place the terminal parallel to the charging coil so that the charging coil is to your right when you're on the terminal.
    2- Pick a walkie and turn it on.
    3- Enter the terminal.
    4- Run the following cell.

In [16]:
waitFor(10)
booster_name = "susie"
last_cmd = ""
on_terminal, flashing, map_updates = False, False, False
msgs = []
walked = False

fix_joystick()

while True:
    if on_terminal:
        if flashing:
            flashBooster(booster_name)
        elif map_updates:
            img = ImageGrab.grab(bbox=sscoords)
            waitFor(0.1)
            pdi_press('esc')
            enemies, items = GetEnemiesAndItems(img)
            msgs = GetStrings(enemies, items)
    
    chat_img = ImageGrab.grab(bbox=chatcoords)

    if on_terminal and map_updates:
         sendMessage(msgs)

    new_cmd = GetChatPromptCode(chat_img)
    
    if new_cmd != last_cmd and new_cmd != "   ":
        print(new_cmd)
        if new_cmd == "Z  ":
            if on_terminal:
                flashing, map_updates = False, True
                walked = False
        elif new_cmd == " Z ":
            if on_terminal:
                flashing, map_updates = True, False
                walked = False
        elif new_cmd == "  Z":
            if on_terminal:
                getDropShip()
                walked = False
        elif new_cmd == "ZZ ":
            if on_terminal:
                pdi_press('esc')
            on_terminal, flashing, map_updates = False, False, False
            charge_walkie()
            walked = False
        elif new_cmd == " ZZ":
            if not on_terminal:
                pdi_press('q')
                walked = False
        elif new_cmd == "Z Z" and not walked:
            if on_terminal:
                pdi_press('esc')
            on_terminal, flashing, map_updates = False, False, False
            walkBack()
            walked = True
        elif new_cmd == "ZZZ":
            pdi_press('e')
            waitFor(1)
            pyperclip.copy("view monitor")
            pyautogui.hotkey('ctrl', 'v')
            waitFor(0.1)
            pdi_press('enter')
            waitFor(1)
            switchTo("SyntaxeAI")
            waitFor(0.1)
            on_terminal = True
            walked = False
        last_cmd = new_cmd

ZZZ
Z  
Z Z
ZZZ


KeyboardInterrupt: 